I discussed earlier that **sequential model** is convenient but can't use all time. For example if we have mnist data having each image 2 output labels, in this case sequential model can't be used.Because it only works for one input to one output mapping. here comes the role of functional model.

In [18]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
#from tensorflow.keras.datasets import mnist

In [19]:
# #for gpu
# physical_devices=tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [20]:
# setting hyperparameters
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [21]:
import pandas as pd
train_df=pd.read_csv("data/train.csv")
test_df=pd.read_csv("data/test.csv")

In [22]:
train_df.head(2)

,Image,first_num,second_num
0,0_00.png,0,0
1,100_00.png,0,0


In [23]:
t_images = os.getcwd() + "data/train_images/" + train_df.iloc[:, 0].values
te_images = os.getcwd() + "data/test_images/" + test_df.iloc[:, 0].values

in the above, getcwd() returns a string, concatenate with the relative pathe of train or test images and then appends each image filename (extracted from the first column of train_df) to form an array of file paths for training images.

In [24]:
print(t_images[:2])
print(len(t_images))
print(train_df.shape)

['c:\\Users\\klikh\\Desktop\\Tensorflowdata/train_images/0_00.png'
 'c:\\Users\\klikh\\Desktop\\Tensorflowdata/train_images/100_00.png']
64000
(64000, 3)


In [25]:
train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [26]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    # image.set_shape((64, 64, 1))
    # label[0].set_shape([])
    # label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels


In [27]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((t_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [28]:
test_dataset = tf.data.Dataset.from_tensor_slices((te_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [29]:
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)
output1 = layers.Dense(10, activation="softmax", name="first_num")(x)
output2 = layers.Dense(10, activation="softmax", name="second_num")(x)
model = keras.Model(inputs=inputs, outputs=[output1, output2])

Notice here we have two output labels from x. It can't be done using sequential model.

In [30]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [31]:
model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)

Epoch 1/5


NotFoundError: Graph execution error:

NewRandomAccessFile failed to Create/Open: c:\Users\klikh\Desktop\Tensorflowdata/train_images/94_54.png : The system cannot find the path specified.
; No such process
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_6029]